# Synthesis S4 (random 4)

## 1. stitch frame

In [ ]:
from utils import stitch_frames
import os
import random

dataset_dir = '/mnt/data1/jiali/avsbench_data/Single-source/s4_data/visual_frames'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        print(split, temp_cate)
        video_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_video in video_list:
            input_video_path = os.path.join(base_dir, temp_cate, temp_video)

            stitch_frames(base_dir, input_video_path, split, save_floder_name = '/avsbench_synthesis_visual_random4/', num_with_audio_and_mask=random.randint(1, 4))

## 2. extract audio log-mel feature

In [1]:
import pickle
import os
from utils import extract_log_mel_features

dataset_dir = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Single-source/s4_data/audio_wav'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        print(split, temp_cate)
        audio_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_audio in audio_list:
            input_audio_path = os.path.join(base_dir, temp_cate, temp_audio)
            log_mel_spectrogram = extract_log_mel_features(input_audio_path, n_mels=128, n_fft=2048, hop_length=512)
            
            # Create the directory for saving if it doesn't exist
            save_path1 = input_audio_path.replace('/audio_wav/', '/audio_log_mel/')
            save_path = save_path1.replace('.wav', '.pkl')
            
            # Save the features
            with open(save_path, 'wb') as f:  # Changed 'w' to 'wb' for binary write
                pickle.dump(log_mel_spectrogram, f)

test mynah_bird_singing
test typing_on_computer_keyboard
test horse_clip-clop
test cat_meowing
test lions_roaring
test chainsawing_trees
test race_car
test playing_violin
test dog_barking
test male_speech
test driving_buses
test helicopter
test cap_gun_shooting
test playing_ukulele
test coyote_howling
test playing_glockenspiel
test playing_tabla
test lawn_mowing
test baby_laughter
test playing_piano
test playing_acoustic_guitar
test female_singing
test ambulance_siren
train mynah_bird_singing
train typing_on_computer_keyboard
train horse_clip-clop
train cat_meowing
train lions_roaring
train chainsawing_trees
train race_car
train playing_violin
train dog_barking
train male_speech
train driving_buses
train helicopter
train cap_gun_shooting
train playing_ukulele
train coyote_howling
train playing_glockenspiel
train playing_tabla
train lawn_mowing
train baby_laughter
train playing_piano
train playing_acoustic_guitar
train female_singing
train ambulance_siren
val mynah_bird_singing
val typi

# Synthesis MS3 (random 4)

## 1. stitch frame

In [2]:
import os
import random
from utils import stitch_frames_ms3
import pandas as pd

def get_split_from_video_id(csv_file, video_id):
    # Read the CSV file into a DataFrame
    df = pd.read_csv(csv_file)
    
    # Search for the row with the given video_id
    row = df[df['video_id'] == video_id]
    
    # If the row is found, return the split value
    if not row.empty:
        return row['split'].values[0]
    else:
        return None


base_dir = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Multi-sources/ms3_data/visual_frames'
csv_file = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Multi-sources/ms3_meta_data.csv'

video_list = os.listdir(base_dir)
for temp_video in video_list:
    input_video_path = os.path.join(base_dir, temp_video)
    split = get_split_from_video_id(csv_file, temp_video) 
    stitch_frames_ms3(base_dir, input_video_path, split, csv_file, save_floder_name = '/avsbench_synthesis_visual_random4/',  num_with_audio_and_mask=random.randint(1, 4))

## 2. extract audio log-mel feature

In [7]:
import pickle
import os
from utils import extract_log_mel_features

dataset_dir = '/mnt/data1/jiali/avsbench_synthesis_visual_random4/Multi-sources/ms3_data/audio_wav'

for split in ['test', 'train', 'val']:
    base_dir = os.path.join(dataset_dir, split)
    entries = os.listdir(base_dir)
    categories = [entry for entry in entries if '.DS_Store' not in entry]
    for temp_cate in categories:
        print(split, temp_cate)
        audio_list = os.listdir(os.path.join(base_dir, temp_cate))
        for temp_audio in audio_list:
            input_audio_path = os.path.join(base_dir, temp_cate, temp_audio)
            log_mel_spectrogram = extract_log_mel_features(input_audio_path, n_mels=128, n_fft=2048, hop_length=512)
            
            # Create the directory for saving if it doesn't exist
            save_path1 = input_audio_path.replace('/audio_wav/', '/audio_log_mel/')
            save_path = save_path1.replace('.wav', '.pkl')
            
            # Save the features
            with open(save_path, 'wb') as f:  # Changed 'w' to 'wb' for binary write
                pickle.dump(log_mel_spectrogram, f)

temp_audio o1bZB4fKv2U_1.wav
temp_audio V9JdDs7RK3c_1.wav
temp_audio xLXcCv45AYg_22.wav
temp_audio kgw1OadfJWA.wav
temp_audio QYvhdbgEJPM_0.wav
temp_audio WjyvwX_nZ6Y_0.wav
temp_audio YYbShzoZWRo_25.wav
temp_audio YYbShzoZWRo_3.wav
temp_audio YYbShzoZWRo_16.wav
temp_audio GJqSBf6p8qg_0.wav
temp_audio FgNwZuMFQwY_0.wav
temp_audio TeLISCmZ_w4_4.wav
temp_audio I8K17mzV-QU_0.wav
temp_audio Vwdib3HWRBI_0.wav
temp_audio CNbSq__q-BY.wav
temp_audio gsAoNfN9Pvk_2.wav
temp_audio SWLG_3suH7w_0.wav
temp_audio UtBQrBLsTQY_3.wav
temp_audio Ox7OpIu5CLI_6.wav
temp_audio nnuN-Zt60TM_3.wav
temp_audio GsfG9ZC8rUU_0.wav
temp_audio pxa8kn8h5ew_0.wav
temp_audio bhDqxWQUIXg_1.wav
temp_audio xGeqjlPz4kw_4.wav
temp_audio FgNwZuMFQwY_1.wav
temp_audio UsTf7brftGg_0.wav
temp_audio BvtaXGFoeRA_0.wav
temp_audio 9xp46AwF9BY_3.wav
temp_audio EnpqqJXs0aY_2.wav
temp_audio ZedzGfGN1tk_1.wav
temp_audio qD-UEwVEDP0_0.wav
temp_audio u4v75o5WxYw_7.wav
temp_audio 3vheOi4y540_1.wav
temp_audio Ox7OpIu5CLI_1.wav
temp_audio _fvp